# **Chessboard Square Detection and Extraction with Computer Vision**
This script leverages computer vision techniques to detect, segment, and extract individual chessboard squares from a given chessboard image. The primary steps involve gradient computation using Sobel filters and line detection using a Hough-like transformation. The program processes the following steps:

*   Gradient Detection: The x and y gradients of the chessboard image are calculated using Sobel filters to highlight the edges along the gridlines of the chessboard.
*   Line Detection: The Hough-like transformation identifies the positions of the horizontal and vertical lines on the board.
*   Line Pruning: Extra or irrelevant lines near the margins are pruned to ensure only valid chessboard gridlines remain.
*   Tile Extraction: Once the gridlines are identified, the script extracts the individual squares from the chessboard and resizes each square to 32x32 pixels for further processing.
*   Image Saving: Each square is saved as a separate image, named according to its position on the board using standard chess notation (e.g., `A1.png`, `B2.png`).

This script automates the task of splitting chessboard images into smaller, labeled tiles, which can be used as training data for machine learning models or for chess-related computer vision tasks. The program can be used with a folder of input images, and it outputs the individual chessboard tiles into a specified directory.

In [ ]:
!pip install opencv-python
import os
import cv2
import numpy as np
import math

# **Gradient Computation, Line Detection, and Pruning for Chessboard Detection**
This section of the code defines several functions that are essential for detecting and refining gridlines on a chessboard image.

**1.   Gradient Computation:**

*   `gradientx(img)` and `gradienty(img)` compute the gradients in the x and y directions, respectively, using Sobel filters with a large kernel size of 31. These gradients highlight edges along the chessboard gridlines.

**2.   Line Matching:**

*   `checkMatch(lineset)` ensures that the detected lines follow a consistent pattern by checking if the differences between consecutive lines are approximately equal, which is expected in a regular chessboard grid.

**3.   Line Pruning:**

*   `pruneLines(lineset, image_dim, margin=20)` removes lines that are too close to the image margins and reduces noise by keeping only valid gridlines that follow the expected pattern of chessboard lines.

**4.   Signal Skeletonization:**

*   `skeletonize_1d(arr)` refines a 1D signal by zeroing out non-maximum values, creating a clear representation of line positions.
These functions work together to detect and validate gridlines on a chessboard image, preparing the data for further processing and segmentation.

In [ ]:
def gradientx(img):
    # Compute gradient in x-direction using larger Sobel kernel
    grad_x = cv2.Sobel(img, cv2.CV_32F, 1, 0, ksize=31)
    return grad_x

def gradienty(img):
    # Compute gradient in y-direction using larger Sobel kernel
    grad_y = cv2.Sobel(img, cv2.CV_32F, 0, 1, ksize=31)
    return grad_y

def checkMatch(lineset):
    linediff = np.diff(lineset)
    x = 0
    cnt = 0
    for line in linediff:
        if abs(line - x) < 5:
            cnt += 1
        else:
            cnt = 0
            x = line
    return cnt == 5

def pruneLines(lineset, image_dim, margin=20):
    # Remove lines near the margins
    lineset = [x for x in lineset if x > margin and x < image_dim - margin]
    if not lineset:
        return lineset
    linediff = np.diff(lineset)
    x = 0
    cnt = 0
    start_pos = 0
    for i, line in enumerate(linediff):
        if abs(line - x) < 5:
            cnt += 1
            if cnt == 5:
                end_pos = i + 2
                return lineset[start_pos:end_pos]
        else:
            cnt = 0
            x = line
            start_pos = i
    return lineset

def skeletonize_1d(arr):
    _arr = arr.copy()
    for i in range(len(_arr) - 1):
        if _arr[i] <= _arr[i + 1]:
            _arr[i] = 0
    for i in range(len(_arr) - 1, 0, -1):
        if _arr[i - 1] > _arr[i]:
            _arr[i] = 0
    return _arr

# **Detecting Chessboard Lines and Extracting Chess Tiles**

In this section, we define two key functions that help in detecting the grid lines of a chessboard from an image and extracting the individual chessboard tiles:

1. **getChessLines**:
    - This function computes horizontal and vertical lines in the image using gradient data (`hdx`, `hdy`). It first applies a Gaussian blur to smooth the thresholded gradient signals.
    - After skeletonizing the signals, it detects the positions of potential chessboard lines.
    - The detected lines are pruned to remove unwanted lines near the image edges, ensuring that only significant lines remain.
    - The function checks whether 7 horizontal and 7 vertical lines are detected, matching the expected grid pattern of an 8x8 chessboard.

2. **getChessTiles**:
    - Once the chessboard lines are identified, this function extracts the individual tiles (squares) from the chessboard.
    - It ensures that the extracted tiles are consistently sized by padding the image where necessary.
    - The function returns a list of 64 tiles, representing the 8x8 grid of the chessboard.

This combination allows us to accurately detect the chessboard structure and divide it into individual squares for further processing.

In [ ]:
def getChessLines(hdx, hdy, hdx_thresh, hdy_thresh, image_shape):
    # Generate Gaussian window
    window_size = 21
    sigma = 8.0
    gausswin = cv2.getGaussianKernel(window_size, sigma, cv2.CV_64F)
    gausswin = gausswin.flatten()
    half_size = window_size // 2

    # Threshold signals
    hdx_thresh_binary = np.where(hdx > hdx_thresh, 1.0, 0.0)
    hdy_thresh_binary = np.where(hdy > hdy_thresh, 1.0, 0.0)

    # Blur signals using convolution with Gaussian window
    blur_x = np.convolve(hdx_thresh_binary, gausswin, mode='same')
    blur_y = np.convolve(hdy_thresh_binary, gausswin, mode='same')

    # Skeletonize signals
    skel_x = skeletonize_1d(blur_x)
    skel_y = skeletonize_1d(blur_y)

    # Find line positions
    lines_x = np.where(skel_x > 0)[0].tolist()
    lines_y = np.where(skel_y > 0)[0].tolist()

    # Prune lines
    lines_x = pruneLines(lines_x, image_shape[1])
    lines_y = pruneLines(lines_y, image_shape[0])

    # Check if lines match expected pattern
    is_match = (len(lines_x) == 7) and (len(lines_y) == 7) and \
               checkMatch(lines_x) and checkMatch(lines_y)

    return lines_x, lines_y, is_match

def getChessTiles(img, lines_x, lines_y):
    stepx = int(round(np.mean(np.diff(lines_x))))
    stepy = int(round(np.mean(np.diff(lines_y))))

    # Pad the image if necessary
    padl_x = 0
    padr_x = 0
    padl_y = 0
    padr_y = 0
    if lines_x[0] - stepx < 0:
        padl_x = abs(lines_x[0] - stepx)
    if lines_x[-1] + stepx > img.shape[1] - 1:
        padr_x = lines_x[-1] + stepx - img.shape[1] + 1
    if lines_y[0] - stepy < 0:
        padl_y = abs(lines_y[0] - stepy)
    if lines_y[-1] + stepy > img.shape[0] - 1:
        padr_y = lines_y[-1] + stepy - img.shape[0] + 1

    img_padded = cv2.copyMakeBorder(img, padl_y, padr_y, padl_x, padr_x, cv2.BORDER_REPLICATE)

    setsx = [lines_x[0] - stepx + padl_x] + [x + padl_x for x in lines_x] + [lines_x[-1] + stepx + padl_x]
    setsy = [lines_y[0] - stepy + padl_y] + [y + padl_y for y in lines_y] + [lines_y[-1] + stepy + padl_y]

    squares = []
    for j in range(8):
        for i in range(8):
            x1 = setsx[i]
            x2 = setsx[i + 1]
            y1 = setsy[j]
            y2 = setsy[j + 1]
            # Adjust sizes to ensure squares are of equal size
            if (x2 - x1) != stepx:
                x2 = x1 + stepx
            if (y2 - y1) != stepy:
                y2 = y1 + stepy
            square = img_padded[y1:y2, x1:x2]
            squares.append(square)
    return squares

# **Processing Chessboard Image and Saving Tiles**

In this section, we define a function `process_image` that takes an image of a chessboard, detects the chessboard lines, slices the board into individual tiles, and saves them as separate image files. Here's a breakdown of the process:

1. **Image Loading and Grayscale Conversion**:
    - The function begins by loading the image from the specified path and converting it to grayscale.

2. **Preprocessing**:
    - The grayscale image is equalized to improve contrast using histogram equalization.
    - The resulting image is normalized to a floating-point representation for subsequent processing.

3. **Gradient Calculation**:
    - Horizontal and vertical gradients are computed using the `gradientx` and `gradienty` functions, which apply large Sobel kernels to emphasize edges.

4. **Hough Transform**:
    - The Hough transform is computed for both the x and y directions by analyzing the gradients. This helps identify the positions of the chessboard lines.

5. **Adaptive Thresholding**:
    - A loop adjusts the threshold values to detect the correct number of chessboard lines (7 horizontal and 7 vertical).
    - The loop terminates when the expected number of lines is found.

6. **Chess Tile Extraction**:
    - If 7 lines in both directions are detected, the chessboard is divided into individual tiles using the `getChessTiles` function.
    - The individual tiles are resized to 32x32 pixels and saved in the specified output directory.

7. **File Naming**:
    - The output directory is named based on the file name (which is assumed to represent the FEN of the chessboard).
    - Each tile is saved with a filename corresponding to its chessboard position (e.g., `A1`, `B2`).

This function processes the image, detects chessboard lines, and saves individual tiles, ready for further analysis or use in chess-related applications.

In [ ]:
def process_image(image_path, output_dir):
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to load image: {image_path}")
        return
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Preprocessing
    equ = cv2.equalizeHist(gray)
    norm_image = equ.astype(np.float32) / 255.0

    # Compute the gradients
    grad_x = gradientx(norm_image)
    grad_y = gradienty(norm_image)

    # Clip the gradients
    Dx_pos = np.clip(grad_x, 0, None)
    Dx_neg = np.clip(-grad_x, 0, None)
    Dy_pos = np.clip(grad_y, 0, None)
    Dy_neg = np.clip(-grad_y, 0, None)

    # Compute the Hough transform
    hough_Dx = (np.sum(Dx_pos, axis=0) * np.sum(Dx_neg, axis=0)) / (norm_image.shape[0] ** 2)
    hough_Dy = (np.sum(Dy_pos, axis=1) * np.sum(Dy_neg, axis=1)) / (norm_image.shape[1] ** 2)

    # Adaptive thresholding
    a = 1
    is_match = False
    lines_x = []
    lines_y = []

    while a < 5:
        threshold_x = np.max(hough_Dx) * (a / 5.0)
        threshold_y = np.max(hough_Dy) * (a / 5.0)

        lines_x, lines_y, is_match = getChessLines(hough_Dx, hough_Dy, threshold_x, threshold_y, norm_image.shape)

        if is_match:
            break
        else:
            a += 1

    if is_match:
        print("7 horizontal and vertical lines found, slicing up squares")
        squares = getChessTiles(gray, lines_x, lines_y)
        print(f"Tiles generated: ({squares[0].shape[0]}x{squares[0].shape[1]}) * {len(squares)}")

        # Extract filename and FEN (assuming filename is FEN)
        img_filename = os.path.basename(image_path)
        fen = os.path.splitext(img_filename)[0]
        img_save_dir = os.path.join(output_dir, fen)

        if not os.path.exists(img_save_dir):
            os.makedirs(img_save_dir)
            print(f"Created dir {img_save_dir}")

        letters = "ABCDEFGH"
        for i, square in enumerate(squares):
            filename = f"{fen}_{letters[i % 8]}{(i // 8) + 1}.png"
            save_path = os.path.join(img_save_dir, filename)
            if i % 8 == 0:
                print(f"#{i}: saving {save_path}...")
            # Resize to 32x32 and save
            resized = cv2.resize(square, (32, 32), interpolation=cv2.INTER_AREA)
            cv2.imwrite(save_path, resized)

    else:
        print(f"No squares to save for {image_path}")

### **Image Processing for Chessboard Square Extraction**

This section of code implements an automated image processing pipeline for extracting chessboard squares from input images. The program specifies input and output directories within Google Colab, allowing users to easily manage image files. The `main()` function scans the specified input folder for image files (PNG, JPG, JPEG), processes each image to identify and extract squares, and saves the resulting tiles into the designated output folder. The processing includes grayscale conversion, gradient computation, adaptive thresholding, and tile resizing, facilitating efficient chessboard analysis and tile generation.

In [ ]:
def main():
    # Specify input and output directories
    input_folder = "train_images"  #@param {type:"string"}
    output_folder = "tiles"        #@param {type:"string"}

    # Create output directory if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Process images in the input folder
    for filename in os.listdir(input_folder):
        if filename.lower().endswith((".png", ".jpg", ".jpeg")):
            image_path = os.path.join(input_folder, filename)
            process_image(image_path, output_folder)

main()